In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Binary_Train

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.nn.functional as F
import random

# 전처리 및 로딩
label_map = {
    '일반대화': 0,
    '협박 대화': 1,
    '갈취 대화': 2,
    '직장 내 괴롭힘 대화': 3,
    '기타 괴롭힘 대화': 4
}

csv_path = '/content/drive/MyDrive/DLthon/merged_dataset_2.csv'
raw_df = pd.read_csv(csv_path)
raw_df['label'] = raw_df['class'].map(label_map)
raw_df.rename(columns={'class': 'original_label', 'conversation': 'text'}, inplace=True)

# 모델 및 데이터셋 정의

tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

class SharedBERT(nn.Module):
    def __init__(self, model_name="beomi/kcbert-base"):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        for p in self.bert.parameters():
            p.requires_grad = False

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

class BinaryClassifier(nn.Module):
    def __init__(self, hidden_size=768):
        super().__init__()
        self.classifier = nn.Linear(hidden_size, 2)

    def forward(self, cls_output):
        return self.classifier(cls_output)

class BinaryClassificationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.texts = dataframe['text'].tolist()
        self.labels = dataframe['label'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.binary_labels = [0 if label == 0 else 1 for label in self.labels]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['label'] = torch.tensor(self.binary_labels[idx], dtype=torch.long)
        item['original_label'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

def train_model(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        cls_output = shared_bert(input_ids, attention_mask)
        logits = model(cls_output)
        loss = F.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# 실행
shared_bert = SharedBERT().to("cuda")
binary_dataset = BinaryClassificationDataset(raw_df, tokenizer)
binary_loader = DataLoader(binary_dataset, batch_size=32, shuffle=True)
binary_head = BinaryClassifier().to("cuda")
optimizer_bin = torch.optim.Adam(binary_head.parameters(), lr=2e-5)

for epoch in range(20):
    loss = train_model(binary_head, binary_loader, optimizer_bin, "cuda")
    print(f"[Binary Epoch {epoch+1}] Loss: {loss:.4f}")

torch.save(binary_head.state_dict(), "/content/drive/MyDrive/DLthon/binary_4.pt")

# 결과 추출만 (파일 저장 제거)
binary_head.eval()
selected_indices = []
incorrect_zero_indices = []

with torch.no_grad():
    for batch_idx, batch in enumerate(binary_loader):
        input_ids = batch['input_ids'].to("cuda")
        attention_mask = batch['attention_mask'].to("cuda")
        original_labels = batch['original_label']

        cls_output = shared_bert(input_ids, attention_mask)
        logits = binary_head(cls_output)
        preds = torch.argmax(logits, dim=1)

        for i, pred in enumerate(preds):
            global_idx = batch_idx * binary_loader.batch_size + i
            true_label = original_labels[i].item()
            if pred.item() == 1 and true_label != 0:
                selected_indices.append(global_idx)
            elif pred.item() == 1 and true_label == 0:
                incorrect_zero_indices.append(global_idx)

print(f"다중 분류 대상으로 넘어갈 샘플 수: {len(selected_indices)}")
print(f"잘못 1로 분류된 일반대화 샘플 수: {len(incorrect_zero_indices)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

[Binary Epoch 1] Loss: 0.5293
[Binary Epoch 2] Loss: 0.4392
[Binary Epoch 3] Loss: 0.3802
[Binary Epoch 4] Loss: 0.3318
[Binary Epoch 5] Loss: 0.2916
[Binary Epoch 6] Loss: 0.2580
[Binary Epoch 7] Loss: 0.2297
[Binary Epoch 8] Loss: 0.2055
[Binary Epoch 9] Loss: 0.1847
[Binary Epoch 10] Loss: 0.1667
[Binary Epoch 11] Loss: 0.1511
[Binary Epoch 12] Loss: 0.1374
[Binary Epoch 13] Loss: 0.1254
[Binary Epoch 14] Loss: 0.1147
[Binary Epoch 15] Loss: 0.1053
[Binary Epoch 16] Loss: 0.0968
[Binary Epoch 17] Loss: 0.0891
[Binary Epoch 18] Loss: 0.0823
[Binary Epoch 19] Loss: 0.0762
[Binary Epoch 20] Loss: 0.0707
✅ 다중 분류 대상으로 넘어갈 샘플 수: 3851
❗ 잘못 1로 분류된 일반대화 샘플 수: 0


In [ ]:
# Multi

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
import pandas as pd

class MultiClassificationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # 일반대화 1000개 + 괴롭힘 전체 구성
        df_0 = dataframe[dataframe['label'] == 0].sample(n=2000, random_state=42)
        df_1to4 = dataframe[dataframe['label'].isin([1, 2, 3, 4])]
        balanced_df = pd.concat([df_0, df_1to4], ignore_index=True)

        self.selected_texts = balanced_df['text'].tolist()
        self.selected_labels = balanced_df['label'].tolist()

    def __len__(self):
        return len(self.selected_texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.selected_texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['label'] = torch.tensor(self.selected_labels[idx], dtype=torch.long)
        return item

class MultiClassifier(nn.Module):
    def __init__(self, hidden_size=768, num_classes=5):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, cls_output):
        return self.classifier(cls_output)

# 학습 및 평가 코드

def train_and_evaluate(dataframe, tokenizer, shared_bert):
    dataset = MultiClassificationDataset(dataframe, tokenizer)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = MultiClassifier().to("cuda")
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

    for epoch in range(50):
        model.train()
        total_loss = 0
        for batch in loader:
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels = batch['label'].to("cuda")

            optimizer.zero_grad()
            cls_output = shared_bert(input_ids, attention_mask)
            logits = model(cls_output)
            loss = F.cross_entropy(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    torch.save(model.state_dict(), "/content/drive/multi_augmentation_6.pt")

    # 평가
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels = batch['label'].to("cpu")

            cls_output = shared_bert(input_ids, attention_mask)
            logits = model(cls_output).to("cpu")
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    print("\n[다중 분류 평가 결과]")
    print(classification_report(all_labels, all_preds, target_names=["일반대화", "협박", "갈취", "직장 내 괴롭힘", "기타 괴롭힘"], digits=4))

    return model

In [1]:
class SharedBERT(nn.Module):
    def __init__(self, model_name="beomi/kcbert-base"):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        for p in self.bert.parameters():
            p.requires_grad = False

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

# class SharedBERT(nn.Module):
#     def __init__(self, model_name="beomi/kcbert-base"):
#         super().__init__()
#         self.bert = BertModel.from_pretrained(model_name)
#         for name, param in self.bert.named_parameters():
#             if 'encoder.layer.9' in name or \
#                 'encoder.layer.10' in name or \
#                     'encoder.layer.11' in name:
#                         param.requires_grad = True
#             else:
#                 param.requires_grad = False

#     def forward(self, input_ids, attention_mask):
#         return self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

NameError: name 'nn' is not defined

In [ ]:
# 토크나이저와 공유 BERT 로드
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

shared_bert = SharedBERT().to("cuda")

# CSV 불러오기 및 라벨 처리
csv_path_ = '/content/drive/MyDrive/DLthon/merged_dataset_4.csv'
df_ = pd.read_csv(csv_path_)

label_map = {
    '일반대화': 0,
    '협박 대화': 1,
    '갈취 대화': 2,
    '직장 내 괴롭힘 대화': 3,
    '기타 괴롭힘 대화': 4
}
df_['label'] = df_['class'].map(label_map)
df_.rename(columns={'class': 'original_label', 'conversation': 'text'}, inplace=True)

# 학습 및 평가
trained_model = train_and_evaluate(df_, tokenizer, shared_bert)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Tapt 시도

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
import pandas as pd

class MultiClassificationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # 일반대화 1000개 + 괴롭힘 전체 구성
        df_0 = dataframe[dataframe['label'] == 0].sample(n=1500, random_state=42)
        df_1to4 = dataframe[dataframe['label'].isin([1, 2, 3, 4])]
        balanced_df = pd.concat([df_0, df_1to4], ignore_index=True)

        self.selected_texts = balanced_df['text'].tolist()
        self.selected_labels = balanced_df['label'].tolist()

    def __len__(self):
        return len(self.selected_texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.selected_texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['label'] = torch.tensor(self.selected_labels[idx], dtype=torch.long)
        return item

class MultiClassifier(nn.Module):
    def __init__(self, hidden_size=768, num_classes=5):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, cls_output):
        return self.classifier(cls_output)

def focal_loss(logits, targets, gamma=2.0, alpha=1.0):
    ce_loss = F.cross_entropy(logits, targets, reduction='none')
    pt = torch.exp(-ce_loss)
    focal = alpha * ((1 - pt) ** gamma) * ce_loss
    return focal.mean()

# 학습 및 평가 코드
def train_and_evaluate(dataframe, tokenizer, shared_bert):
    from copy import deepcopy

    dataset = MultiClassificationDataset(dataframe, tokenizer)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = MultiClassifier().to("cuda")
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

    best_loss = float('inf')
    best_model_state = deepcopy(model.state_dict())
    patience = 4
    patience_counter = 0

    for epoch in range(30):
        model.train()
        total_loss = 0

    for batch in loader:
        input_ids = batch['input_ids'].to("cuda")
        attention_mask = batch['attention_mask'].to("cuda")
        labels = batch['label'].to("cuda")

        optimizer.zero_grad()
        output = shared_bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0, :]  # [CLS] 벡터 추출
        logits = model(cls_output)

        # focal loss for label==4 (기타 괴롭힘), cross-entropy for others
        if (labels == 4).any():
            focal_mask = labels == 4
            ce_mask = ~focal_mask

            focal_loss_val = (
                focal_loss(logits[focal_mask], labels[focal_mask])
                if focal_mask.sum() > 0 else 0
            )
            ce_loss_val = (
                F.cross_entropy(logits[ce_mask], labels[ce_mask])
                if ce_mask.sum() > 0 else 0
            )
            loss = focal_loss_val + ce_loss_val
        else:
            loss = F.cross_entropy(logits, labels)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 최적 모델 저장
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    else:
        print("Warning: No best model was saved. Using the last trained model.")

    torch.save(model.state_dict(), "/content/drive/MyDrive/DLthon/tapt_multi_1.pt")

    # 평가
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels = batch['label'].to("cpu")

            cls_output = shared_bert(input_ids, attention_mask)
            logits = model(cls_output).to("cpu")
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    print("\n[다중 분류 평가 결과]")
    print(classification_report(
        all_labels,
        all_preds,
        target_names=["일반대화", "협박", "갈취", "직장 내 괴롭힘", "기타 괴롭힘"],
        digits=4
    ))

    return model

In [ ]:
class SharedBERT(nn.Module):
    def __init__(self, model_name="beomi/kcbert-base"):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        for p in self.bert.parameters():
            p.requires_grad = False

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

In [ ]:
# 토크나이저와 공유 BERT 로드
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

shared_bert = BertModel.from_pretrained("/content/drive/MyDrive/DLthon/tapt_kcbert").to('cuda')

# CSV 불러오기 및 라벨 처리
csv_path_ = '/content/drive/MyDrive/DLthon/merged_dataset_2.csv'
df_ = pd.read_csv(csv_path_)

label_map = {
    '일반대화': 0,
    '협박 대화': 1,
    '갈취 대화': 2,
    '직장 내 괴롭힘 대화': 3,
    '기타 괴롭힘 대화': 4
}
df_['label'] = df_['class'].map(label_map)
df_.rename(columns={'class': 'original_label', 'conversation': 'text'}, inplace=True)

# 학습 및 평가
trained_model = train_and_evaluate(df_, tokenizer, shared_bert)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/DLthon/tapt_kcbert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: linear(): argument 'input' (position 1) must be Tensor, not BaseModelOutputWithPoolingAndCrossAttentions

In [ ]:
# Focal Loss 시도

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
import pandas as pd
from copy import deepcopy

# focal loss 함수
def focal_loss(logits, targets, gamma=2.0, alpha=1.0):
    ce_loss = F.cross_entropy(logits, targets, reduction='none')
    pt = torch.exp(-ce_loss)
    focal = alpha * ((1 - pt) ** gamma) * ce_loss
    return focal.mean()

# 데이터셋 클래스
class MultiClassificationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.tokenizer = tokenizer
        self.max_length = max_length

        df_0 = dataframe[dataframe['label'] == 0].sample(n=1500, random_state=42)
        df_1to4 = dataframe[dataframe['label'].isin([1, 2, 3, 4])]
        balanced_df = pd.concat([df_0, df_1to4], ignore_index=True)

        self.selected_texts = balanced_df['text'].tolist()
        self.selected_labels = balanced_df['label'].tolist()

    def __len__(self):
        return len(self.selected_texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.selected_texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['label'] = torch.tensor(self.selected_labels[idx], dtype=torch.long)
        return item

# 분류기
class MultiClassifier(nn.Module):
    def __init__(self, hidden_size=768, num_classes=5):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, cls_output):
        return self.classifier(cls_output)

# 학습 및 평가 함수
def train_and_evaluate(dataframe, tokenizer, shared_bert):
    dataset = MultiClassificationDataset(dataframe, tokenizer)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = MultiClassifier().to("cuda")
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

    best_loss = float('inf')
    best_model_state = deepcopy(model.state_dict())
    patience = 4
    patience_counter = 0

    for epoch in range(30):
        model.train()
        total_loss = 0

        for batch in loader:
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels = batch['label'].to("cuda")

            optimizer.zero_grad()
            output = shared_bert(input_ids=input_ids, attention_mask=attention_mask)
            cls_output = output.last_hidden_state[:, 0, :]  # [CLS] 벡터
            logits = model(cls_output)

            if (labels == 4).any():
                focal_mask = labels == 4
                ce_mask = ~focal_mask

                focal_loss_val = (
                    focal_loss(logits[focal_mask], labels[focal_mask])
                    if focal_mask.sum() > 0 else 0
                )
                ce_loss_val = (
                    F.cross_entropy(logits[ce_mask], labels[ce_mask])
                    if ce_mask.sum() > 0 else 0
                )
                loss = focal_loss_val + ce_loss_val
            else:
                loss = F.cross_entropy(logits, labels)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        print(f"[Epoch {epoch+1}] Loss: {avg_loss:.4f}")

        if avg_loss < best_loss - 1e-4:
            best_loss = avg_loss
            best_model_state = deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"  ↳ No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("  ↳ Early stopping triggered.")
                break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    else:
        print("Warning: Best model was not saved. Using last model state.")

    torch.save(model.state_dict(), "/content/drive/MyDrive/DLthon/tapt_multi_2.pt")

    # 평가
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels = batch['label'].to("cpu")

            output = shared_bert(input_ids=input_ids, attention_mask=attention_mask)
            cls_output = output.last_hidden_state[:, 0, :]
            logits = model(cls_output).to("cpu")
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    print("\n[다중 분류 평가 결과]")
    print(classification_report(
        all_labels,
        all_preds,
        target_names=["일반대화", "협박", "갈취", "직장 내 괴롭힘", "기타 괴롭힘"],
        digits=4
    ))

    return model

In [ ]:
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/DLthon/tapt_kcbert")
shared_bert = BertModel.from_pretrained("/content/drive/MyDrive/DLthon/tapt_kcbert").to('cuda')

csv_path_ = '/content/drive/MyDrive/DLthon/merged_dataset_4.csv'
df_ = pd.read_csv(csv_path_)

label_map = {
    '일반대화': 0,
    '협박 대화': 1,
    '갈취 대화': 2,
    '직장 내 괴롭힘 대화': 3,
    '기타 괴롭힘 대화': 4
}
df_['label'] = df_['class'].map(label_map)
df_.rename(columns={'class': 'original_label', 'conversation': 'text'}, inplace=True)

train_and_evaluate(df_, tokenizer, shared_bert)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/DLthon/tapt_kcbert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Epoch 1] Loss: 1.3284
[Epoch 2] Loss: 0.9535
[Epoch 3] Loss: 0.6864
[Epoch 4] Loss: 0.5127
[Epoch 5] Loss: 0.3764
[Epoch 6] Loss: 0.2911
[Epoch 7] Loss: 0.2285
[Epoch 8] Loss: 0.1824
[Epoch 9] Loss: 0.1483
[Epoch 10] Loss: 0.1177
[Epoch 11] Loss: 0.1000
[Epoch 12] Loss: 0.0845
[Epoch 13] Loss: 0.0659
[Epoch 14] Loss: 0.0602
[Epoch 15] Loss: 0.0519
[Epoch 16] Loss: 0.0433
[Epoch 17] Loss: 0.0405
[Epoch 18] Loss: 0.0343
[Epoch 19] Loss: 0.0324
[Epoch 20] Loss: 0.0280
[Epoch 21] Loss: 0.0241
[Epoch 22] Loss: 0.0220
[Epoch 23] Loss: 0.0196
[Epoch 24] Loss: 0.0174
[Epoch 25] Loss: 0.0177
  ↳ No improvement. Patience: 1/4
[Epoch 26] Loss: 0.0151
[Epoch 27] Loss: 0.0143
[Epoch 28] Loss: 0.0131
[Epoch 29] Loss: 0.0121
[Epoch 30] Loss: 0.0103

[다중 분류 평가 결과]


ValueError: Number of classes, 1, does not match size of target_names, 5. Try specifying the labels parameter